In [1]:
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
# Create outputs dir 
dir = r'..\..\..\outputs\compare\RTP\standard'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

# Total Population

In [6]:
# Total Population
# Load Network Summary sheet from network_summary_detailed
df = pd.read_csv(os.path.join(relative_path,'agg/dash',r'pptyp_county.csv'))
df = df.groupby('person_county').sum()[['psexpfac']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'psexpfac': 'Population', 'person_county': 'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','1_total_population.csv'),
         index=False)
HTML(df.to_html(index=False))

County,Population
King,"2,149,234"
Kitsap,"257,159"
Pierce,"851,709"
Snohomish,"795,052"
Total,"4,053,154"


# Total Employment
By Workplace Location

In [7]:
df = pd.read_csv(os.path.join(relative_path,'agg/dash',r'person_worker_type.csv'))
df = df.groupby('person_work_county').sum()[['psexpfac']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'psexpfac': 'Population', 'person_work_county': 'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','2_total_employment.csv'))
HTML(df.to_html(index=False))

County,Population
King,"1,225,841"
Kitsap,"88,522"
Pierce,"248,609"
Snohomish,"232,917"
Total,"1,795,889"


# Vehicle Miles Traveled
Includes truck and external trips

In [8]:
df_network = load_network_summary(os.path.join(relative_path,'network',r'network_results.csv'))
df = df_network.groupby('county').sum()[['VMT']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','3_vmt.csv'))
HTML(df.to_html(index=False))

County,VMT
King,"47,482,181"
Kitsap,"4,664,734"
Pierce,"18,471,740"
Snohomish,"16,813,030"
Total,"87,431,685"


# Vehicle Hours of Delay

In [9]:
df = df_network.groupby('county').sum()[['VHT']]
df.loc['Total',:] = df.sum()
df = df.reset_index()
df.rename(columns={'county':'County'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','4_vht.csv'))
HTML(df.to_html(index=False))

County,VHT
King,"1,555,260"
Kitsap,"134,944"
Pierce,"577,850"
Snohomish,"505,727"
Total,"2,773,780"


# Transit Boardings

In [10]:
df = pd.read_csv(os.path.join(relative_path,'transit',r'daily_boardings_by_agency.csv'), index_col='agency_name')
df.loc['Total',:] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name':'Agency', 'boardings': 'Daily Boardings'}, inplace=True)
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','5_transit_boardings.csv'))
HTML(df.to_html(index=False))

Agency,Daily Boardings
King County Metro,"397,594"
Sound Transit,"157,231"
Pierce Transit,"33,931"
Community Transit,"33,773"
Kitsap Transit,"14,471"
Washington Ferries,"12,285"
Everett Transit,"5,340"
Total,"654,626"


# Emissions
Daily Tons

In [11]:
df = pd.read_csv(os.path.join(relative_path,'emissions',r'emissions_summary.csv'))
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'}, inplace=True)
df = df[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent'])]
df.index.name = None
df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df.to_csv(os.path.join(os.getcwd(), r'..\..\..\outputs\compare\RTP\standard','6_emissions.csv'))
df

,Start,Intrazonal,Interzonal,Total Daily
CO,186,2.0,299,487
CO2 Equivalent,"2,568",215,"40,711","43,494"
NOx,10.6,0.2,68.6,79.5
PM10 Total,0.5,0.0,5.8,6.3
PM25 Total,0.4,0.0,1.8,2.3
